# <center>常用的采样方法</center>

最近在看给予蒙特卡罗的强化学习方法，正好组会上学姐汇报“A stochastic approximation algorithm for maximum-likelihood estimation with incomplete data”论文时简单介绍了MC和Gibbs采样，所以趁着这次机会把相关采样方法做个笔记。

常用的采样方法有两类。第一类是指定一个已知的概率分布$p(x)$用于采样，指定的采样概率分布称为提议分布。这类采样方法包括拒绝采样和重要性采样。只适用于低维情况，高维情况常采用第二类采样方法，即马尔科夫链蒙特卡罗方法。

### 为何要抽样？

生活中，我们常常需要抽取样本来对总体的分布进行估计。但是当随机变量的分布非常复杂时，无法利用解析的方法产生用于逼近期望的样本。在计算机模拟时，我们所说的抽样，其实是指从一个概率分布中生成观察值（observations）的方法。而这个分布通常是由其概率密度函数（PDF）来表示的。而且，即使在已知PDF的情况下，让计算机自动生成观测值也不是一件容易的事情。从本质上来说，计算机只能实现对均匀分布（Uniform distribution）的采样。

具体来说，我们可能要面对的问题包括：

1.计算机只能实现对均匀分布的采样，但我们仍然可以在此基础上对更为复杂的分布进行采样，那具体该如何操作呢？  
2.随机分布的某些数字特征可能需要通过积分的形式来求解，但是某些积分可能没有（或者很难求得）解析解，彼时我们该如何处理呢？  
3.在贝叶斯推断中，后验概率的分布是正⽐于先验和似然函数之积的，但是先验和似然函数的乘积形式可能相对复杂，我们又该如何对这种形式复杂的分布进行采样呢？

# 拒绝采样（Rejection Sampling）

假设我们想对PDF为$f(x)$的函数进行采样，常规思路由于其积分，即分布函数的取值范围在0-1中，又因为分布函数的分布函数服从均匀分布（$P(X\leq F^{-1}(t))=F(F^{-1}(t))=t,\forall t\in [0,1]\implies F(x)\sim U[0,1]$），故可取均匀分布$i\sim U(0,1)$，再求其逆函数$x_i=F^{-1}(i)$。可是由于逆函数很难求，这种方法有时并不可取。
  
拒绝采样适用于由于种种原因（例如这个函数很复杂），对其进行采样是相对困难的。但是另外一个PDF为$q(x)$的函数则相对容易采样，如均匀分布、正态分布等，我们可通过对其乘上一个常数$m$，使其可以将$f(x)$罩住，即找到一个$q(x)$满足$\left\{\begin{align}&q(x)的CDF可求\\&f(x)\leq mq(x)\end{align}\right.\quad$这样我们就可以从来自于分布$q(x)$的样本中抽样来自$f(x)$的样本，具体来讲：  
从$U(0,1)$中选取$i$（原因：分布函数的分布函数服从均匀分布），求其逆函数$x_i$，若取$q(x)$为$U(x)$，则$x_i=Q^{-1}(x)=\dfrac{mx}{b-a}$，当$x_i<p=\dfrac{f(x)}{mq(x)}$时，接受样本$x_i$，重复上述过程，直到产生足够数量的样本点。通常情况下为提高效率，会使满足$f(x)\leq mq(x)$的$m$尽可能的小。

<img style="float: center;" src="1.jpg" width="60%" height="100%">

# 重要性采样
以后再更  
当维度比较高时，拒绝采样和重要性采样不适用。MCMC与其原理有本质区别。拒绝采样和重要性采样利用提议分布产生样本点，当维度很高时难以找到合适的提议分布，采样效率差。

# 蒙特卡洛法（MCMC方法）

$\quad$MCMC不需要提议分布，只需要一个随机样本点，下一个样本点会由当前的样本点产生，如此循环源源不断产生很多样本点。最终，这些样本点服从目标分布。  
$\quad$如何通过当前样本点产生下一个样本点，并保证如此产生的样本服从原目标分布呢？  
其定理是：目标分数为马氏链平稳分布。简单而言，该目标分布存在一个转移概率矩阵$P$,且该转移概率满足：  
$\pi(j)=\sum_{i=0}^\infty\pi(i)P_{ij}$；$\pi$是方程$\pi P=\pi$的唯一非负解。  
$\quad$当转移矩阵$P$满足上述条件时，从任意初始分布$\pi_0$出发，经过一段时间迭代，分布$\pi_t$都会收敛到目标分布$\pi$。因此，假设我们已经知道了满足条件的状态转移概率矩阵$P$，那么我们只要给出任意一个初始状态$x_0$，则可以得到一个转移序列$x_0$，$x_1$，$\dots$，$x_n$，$x_{n+1}$，$\dots$。如果该马氏链在第$n$步已经收敛到目标分布$\pi$，那么我们就得到了服从目标分布$\pi$的样本$x_n$，$x_{n+1}$，$\dots$。  
$\quad$现在问题转化为寻找与目标分布相对应的转移概率$P$，那么如何构造转移概率呢？  
$\quad$转移概率$P$和分布$\pi(x)$应该满足细致平稳条件。所谓细致平稳条件，即  
$$\pi(i)P_{ij}=\pi(j)P_{ji}\quad\forall i,j$$  
$\quad$接下来，如何利用细致平稳条件构造转移概率呢？  
$\quad$我们可以这样考虑：加入已有的一个转移矩阵$Q$的马氏链，这样任意选的转移矩阵通常情况下并不满足细致平衡条件，也就是  
$$p(i)q(i,j)\neq p(j)q(j,i)$$
$\quad$既然不满足，我们就可以改造$q(i,j)$，使之满足。改造的方法是加入一项$\alpha(i,j)$使得  
$$p(i)q(i,j)\alpha(i,j)=p(j)q(j,i)\alpha(j,i)$$  
$\quad$问题是如何取$\alpha(i,j)$呢？一个简单的想法是利用式子的对称性，即  
$$\alpha(i,j)=p(j)q(j,i),\alpha(j,i)=p(i)q(i,j)$$
其中$\alpha(i,j)$被称为接受率。

<img style="float: center;" src="2.jpg" width="60%" height="100%">

为了提高接受率，使得样本多样化，MCMC的第5行接受率通常可改写为$\alpha(x_t,y)=\min\left\{\dfrac{p(y)q(x_t|y)}{p(x_t)p(y|x_t)},1\right\}$，采样这种接受率的算法称为**Metropolis-Hastings算法**。

# 吉布斯采样（Gibbs Sampling）

从二维的数据分布开始，由前面细致平衡条件，根据下图推导可得：$$\pi(A)P(A\rightarrow A')=\pi(A')P(A'\rightarrow A)$$

<img style="float: center;" src="3.jpg" width="60%" height="100%">

利用上一节找到的状态转移矩阵，我们就得到了二维Gibbs采样，这个采样需要两个维度之间的条件概率。具体过程如下：

1. 输入平稳分布$\pi(𝑥_1,𝑥_2)$，设定状态转移次数阈值$𝑛_1$，需要的样本个数$𝑛_2$
2. 随机初始化初始状态值$𝑥^{(0)}1$和$𝑥^{(0)}2$
3. for $𝑡=0$ to $𝑛_1+𝑛2−1$: 

- 从条件概率分布$𝑃(𝑥_2|𝑥^{(𝑡)}1)$中采样得到样本$𝑥^{(𝑡+1)}_2$
- 从条件概率分布$𝑃(𝑥_1|𝑥^{(𝑡+1)}2)$中采样得到样本$𝑥^{(𝑡+1)}_1$  
不停轮换坐标轴，样本集${(𝑥^{(𝑛_1)}_1,𝑥^{(𝑛_1)}_2),(𝑥^{(𝑛_1+1)}_1,𝑥^{(𝑛_1+1)}_2),...,(𝑥^{(𝑛_1+𝑛_2−1)}_1,𝑥^{(𝑛_1+𝑛_2−1)}_2)}$即为我们需要的平稳分布对应的样本集。

$\quad$整个采样过程中，我们通过轮换坐标轴，采样的过程为：
$$(𝑥^{(1)}_1,𝑥^{(1)}_2)\rightarrow(𝑥^{(1)}_1,𝑥^{(2)}_2)\rightarrow(𝑥^{(2)}_1,𝑥^{(2)}_2)\rightarrow\dots\rightarrow(𝑥^{(𝑛_1+𝑛_2−1)}_1,𝑥^{(𝑛_1+𝑛_2−1)}_2)$$
用下图可以很直观的看出，采样是在两个坐标轴上不停的轮换的。当然，坐标轴轮换不是必须的，我们也可以每次随机选择一个坐标轴进行采样。不过常用的Gibbs采样的实现都是基于坐标轴轮换的。
<img style="float: center;" src="4.jpg" width="35%" height="35%">

$\quad$多维的时候也是成立的。比如一个n维的概率分布$\pi(𝑥_1,𝑥_2,\dots,𝑥_𝑛)$，我们可以通过在$n$个坐标轴上轮换采样，来得到新的样本。对于轮换到的任意一个坐标轴$𝑥_𝑖$上的转移，马尔科夫链的状态转移概率为$𝑃(𝑥_𝑖|𝑥_1,𝑥_2,\dots,𝑥_{𝑖−1},𝑥_{𝑖+1},\dots,𝑥_𝑛)$，即固定$𝑛−1$个坐标轴，在某一个坐标轴上移动。

### 参考资料
- B站asia1987老师的视频讲解
- 深入浅出强化学习原理入门 郭宪、方勇纯编著